<a href="https://colab.research.google.com/github/souroveskb/ml-resources/blob/main/Pytorch-Tutorials/fc_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
#create Fully Connected network
class NN(nn.Module):
  def __init__(self, input_size, num_classes): #input-->784
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

#Initialising devices
'Cuda' if it is available 'Cpu' if not

In [4]:
#initialise devices
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#Hyperparameters
The values which needs to be predefined and are not included in the learning process(not subject to changes during training or deployment)

In [5]:
#hyperparameters
input_size = 784
num_classes = 10  #mnist data for 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 5

We use existing datasets from pytorch datasets and operate on the MNIST number dataset.
Create a root directory to store the data. Download if not already downloaded.
Transform the Data to tensor as the loaded data would come as a numpy array.

In [7]:
#laoding the Data
train_dataset = datasets.MNIST(root= 'dataset/', train=True, download=True,
               transform= transforms.ToTensor())
train_loader = DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle= True)
test_dataset = datasets.MNIST(root= 'dataset/', train=False, download=True,
               transform= transforms.ToTensor())
test_loader = DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle= True)

100%|██████████| 9912422/9912422 [00:00<00:00, 118692453.33it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 43185630.60it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32098410.35it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18143360.73it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



#Initialize the Network

In [9]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

#Loss function and Optimizer initialisation

In [11]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Train the network


In [21]:
for epoch in range(num_epochs): #num_epochs
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    #get to batch_size, 784
    data = data.reshape(data.shape[0], -1)

    #forward
    scores = model(data)
    loss = loss_function(scores, targets)

    #backward
    optimizer.zero_grad()
    loss.backward()

    #gradient descent
    optimizer.step()


#Check the accuracy

In [18]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)
      scores = model(x)

      #taking the maximum of 10 digits of 64x10
      _, predictions = scores.max(1) #values and index are given back but we only need the index
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples) *100:.2f}')

  model.train() #if we want to check during the training process

In [22]:
#Check the accuracy
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59091/60000 with accuracy 98.48
Checking accuracy on test data
Got 9715/10000 with accuracy 97.15


epoch *5*
Checking accuracy on training data
Got 58397/60000 with accuracy 97.33
Checking accuracy on test data
Got 9653/10000 with accuracy 96.53